In [1]:
from pathlib import Path
from datetime import datetime
from time import time
import cv2
import torch
import numpy as np
import pickle
from scipy.spatial.distance import cosine
from mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input


Using TensorFlow backend.


In [2]:
# BASE_DIR = Path(__file__).parent.parent
# BASE_DIR = Path.cwd().
DRIVE_NAME = "F:\\"
DATA_DIR = Path(DRIVE_NAME).joinpath('data')
IMAGE_DIR = DATA_DIR.joinpath('images')
DUMP_DIR = DATA_DIR.joinpath('dump')
MODEL_DIR = DATA_DIR.joinpath('models')
# for c in DATA_DIR.iterdir(): print(c)


In [3]:
mtcnn_face_detector = MTCNN()
# vgg_embedding_extractor = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
vgg_embedding_extractor = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
print('Inputs: %s' % vgg_embedding_extractor.inputs)
print('Outputs: %s' % vgg_embedding_extractor.outputs)


Inputs: [<tf.Tensor 'input_4:0' shape=(None, 224, 224, 3) dtype=float32>]
Outputs: [<tf.Tensor 'global_average_pooling2d_17/Mean:0' shape=(None, 2048) dtype=float32>]


In [4]:
test_dir = IMAGE_DIR.joinpath('2010-09-27')

images_list = []

embeddings_list = []
# index = 0
for image_path in test_dir.iterdir():
    image_dict = {
        'path': image_path,
        'face_list': [],
    }
#     print(image_path)
    # index += 1
    image = cv2.imread(str(image_path))
#     print(image.shape)
    
    faces = mtcnn_face_detector.detect_faces(image)
#     print(len(faces))
    face_index = 0
    for face_dict in faces:
        
        
        face_index += 1
        confidence = face_dict['confidence']
#         print(confidence)
        if confidence < 0.5:
            continue
        (x, y, width, height) = face_dict['box']
        
        
        
        # Get face image from bounding box and preprocess
        face_image = image[y:y + height, x:x + width]
        face_image = cv2.resize(face_image, (224, 224))
        face_image_window = '{}'.format(face_index)
        cv2.imshow(face_image_window, face_image)
        cv2.moveWindow(face_image_window, 224*(face_index - 1), 0)
        
        # Draw bounding box
        image = cv2.rectangle(
            image,
            (x, y),
            (x + width, y + height),
            (0, 255, 0),
            2,
        )
        
        samples = preprocess_input(np.expand_dims(face_image.astype('float32'), axis=0), version=2)
        embedding = vgg_embedding_extractor.predict(samples)
        
        face_data_dict = {
            'source_image_path': image_path,
            'mtcnn_extraction': face_dict,
            'vgg_extraction': embedding,
        }
        image_dict['face_list'].append(face_data_dict)
    images_list.append(image_dict)
    cv2.imshow('test image', image)
    cv2.moveWindow('test image', 0, 224)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    



InvalidImage: Image not valid.

In [ ]:
with open(test_dir.joinpath(test_dir.name + '.pkl'), 'wb') as save_path:
    pickle.dump(images_list, save_path)
    
    
# np.linalg.norm(embeddings_list[0] - embeddings_list[0], ord=2)
# cosine(embeddings_list[6], embeddings_list[7])

# with open(test_dir.joinpath(test_dir.name + '.pkl'), 'rb') as load_path:
#     asdf = pickle.load(load_path)